In [1]:
import numpy as np
import seaborn as sb
import pandas as pd
import os
import xml.etree.ElementTree as ET
import json
import csv
import string


In [3]:
datos ='TrainingDatasets/TREC-2017-LiveQA-Medical-Train-1.xml'
datos1 ='TrainingDatasets/TREC-2017-LiveQA-Medical-Train-2.xml'

In [3]:
def combine_datasets(datos, datos1, nomb_archivo: str):
    # Parsea los documentos XML
    tree1 = ET.parse(datos)
    tree2 = ET.parse(datos1)

    root1 = tree1.getroot()
    root2 = tree2.getroot()

    # Supongamos que queremos agregar todos los elementos de root2 a root1
    for child in root2:
        root1.append(child)

    # Escribe el nuevo documento XML combinado
    tree1.write(f'{nomb_archivo}.xml')

In [9]:
combine_datasets(datos, datos1,'datasets_comb')

In [12]:
def get_xml_to_json(datos):
    # Parsear el archivo XML
    tree = ET.parse(datos)
    root = tree.getroot()

    # Listas para almacenar los datos
    data = []

    # Iterar sobre los nodos NLM-QUESTION
    for question in root.findall('.//NLM-QUESTION'):
        question_id = question.attrib.get('questionid') or question.attrib.get('qid')
        subject = question.find('SUBJECT').text if question.find('SUBJECT') is not None else None  # Añadir 'subject' si existe
        message = question.find('MESSAGE').text
        focus_list = [subquestion.find('.//FOCUS').text for subquestion in question.findall('.//SUB-QUESTION')]
        type_info_list = [subquestion.find('.//TYPE').text for subquestion in question.findall('.//SUB-QUESTION')]
        answer_list = [answer.text for subquestion in question.findall('.//SUB-QUESTION') for answer in subquestion.findall('.//ANSWER')]
        if subject is None:
            subject = ','
        if message is None:
            message = subject
        
        answer_text = ''
        focus = ''
        type_info = ''
        for f in focus_list:
            focus = focus + f'{f},'
        
        for info in type_info_list:
            type_info = type_info + f'{info}'
        
        for answer in answer_list:
            answer_text = answer_text + f'{answer}'
        
        data.append({'question_id': question_id,
                            'question_text': message,
                            'subject': subject,
                            'focus': focus,
                            'type': type_info,                            
                            'answer_text': answer_text})


    #Crear archivo JSON
    with open('datos.json', 'w') as json_file:
        json.dump(data, json_file, indent=4)
        

In [13]:

get_xml_to_json('datasets_comb.xml')


In [2]:
 # Parsear el archivo XML
tree = ET.parse('datasets_comb.xml')
root = tree.getroot()

# Listas para almacenar los datos
data = []

# Iterar sobre los nodos NLM-QUESTION
for question in root.findall('.//NLM-QUESTION'):
    question_id = question.attrib.get('questionid') or question.attrib.get('qid')
    subject = question.find('SUBJECT').text if question.find('SUBJECT') is not None else None  # Añadir 'subject' si existe
    message = question.find('MESSAGE').text
    focus_list = [subquestion.find('.//FOCUS').text for subquestion in question.findall('.//SUB-QUESTION')]
    type_info_list = [subquestion.find('.//TYPE').text for subquestion in question.findall('.//SUB-QUESTION')]
    answer_list = [answer.text for subquestion in question.findall('.//SUB-QUESTION') for answer in subquestion.findall('.//ANSWER')]
    if subject is None:
        subject = ','
    if message is None:
        message = subject
    
    answer_text = ''
    focus = ''
    type_info = ''
    for f in focus_list:
        focus = focus + f'{f},'
        
    for info in type_info_list:
        type_info = type_info + f' {info}'
        
    for answer in answer_list:
        answer_text = answer_text + f'{answer}'
        
    data.append({'question_id': question_id,
                        'question_text': message,
                        #'subject': subject,
                        #'focus': focus,
                        'type': type_info,                            
                        'answer_text': answer_text})

# Crear DataFrame
df = pd.DataFrame(data)

df

,question_id,question_text,type,answer_text
0,Q1,Literature on Cardiac amyloidosis. Please let...,information,Cardiac amyloidosis is a disorder caused by de...
1,Q2,Migraine seems to be a spectrum of conditions ...,treatment,There is no specific cure for migraine headach...
2,Q3,DO I USE PYRIDOXINE TABLETS EVEN IF IM PREGNANT?,contraindication,"Before taking pyridoxine, tell your doc..."
3,Q4,i have lymphoma what causes cramp after chemo ...,cause,Muscle cramps are common and often occur when ...
4,Q5,I wonder of new research and testing on macula...,treatment,These resources address the diagnosis or manag...
...,...,...,...,...
441,316.0,If you should contact MRSA and take antibiotic...,Diagnosis,"Occasionally, a person who has been treated fo..."
442,317.0,What if you do not have surgery to remove a pa...,Prognosis,Your specialist will be able to advise you of ...
443,318.0,Thank for your information on my earlier quest...,Treatment,Your doctor may order tests to help determine ...
444,319.0,I have numbness/tingling in my lower right arm...,Treatment,Home Care Your doctor should find and treat t...


In [9]:
# Parsear el archivo XML
tree = ET.parse('datasets_comb.xml')
root = tree.getroot()

# Listas para almacenar los datos
data = []

# Iterar sobre los nodos NLM-QUESTION
for question in root.findall('.//NLM-QUESTION'):
    question_id = question.attrib.get('questionid') or question.attrib.get('qid')
    subject = question.find('SUBJECT').text if question.find('SUBJECT') is not None else None  # Añadir 'subject' si existe
    message = question.find('MESSAGE').text
    if subject is None:
        subject = ','
    if message is None:
        message = subject
    # Iterar sobre los nodos SUB-QUESTION dentro de cada NLM-QUESTION
    for subquestion in question.findall('.//SUB-QUESTION'):
        #subquestion_id = subquestion.attrib.get('subqid')
        focus = subquestion.find('.//FOCUS').text
        type_info = subquestion.find('.//TYPE').text
            
            # Iterar sobre los nodos ANSWER dentro de cada SUB-QUESTION
        for answer in subquestion.findall('.//ANSWER'):
            #answer_id = answer.attrib.get('answerid')
            answer_text = answer.text
                
            data.append({'question_id': question_id,
                            'message': message,
                            'focus': focus,
                            'type': type_info,
                            'answer_text': answer_text})
   
df2 = pd.DataFrame(data)
df2

,question_id,message,focus,type,answer_text
0,Q1,Literature on Cardiac amyloidosis. Please let...,cardiac amyloidosis,information,Cardiac amyloidosis is a disorder caused by de...
1,Q1,Literature on Cardiac amyloidosis. Please let...,cardiac amyloidosis,information,"The term ""amyloidosis"" refers not to a single ..."
2,Q2,Migraine seems to be a spectrum of conditions ...,migraine,treatment,There is no specific cure for migraine headach...
3,Q2,Migraine seems to be a spectrum of conditions ...,migraine,treatment,There is no absolute cure for migraine since i...
4,Q3,DO I USE PYRIDOXINE TABLETS EVEN IF IM PREGNANT?,pyridoxine,contraindication,"Before taking pyridoxine, tell your doc..."
...,...,...,...,...,...
629,316.0,If you should contact MRSA and take antibiotic...,MRSA,Diagnosis,"Occasionally, a person who has been treated fo..."
630,317.0,What if you do not have surgery to remove a pa...,intraductal papilloma,Prognosis,Your specialist will be able to advise you of ...
631,318.0,Thank for your information on my earlier quest...,excessive tears,Treatment,Your doctor may order tests to help determine ...
632,319.0,I have numbness/tingling in my lower right arm...,numbness/tingling,Treatment,Home Care Your doctor should find and treat t...


In [7]:
df = pd.read_csv('data_test.csv')


In [3]:
df

,question_id,subject,question_text,nist_paraphrase,nlm_summary,focus,type_info,answer_list
0,TQ1,Noonan syndrome,What are the references with noonan syndrome a...,What is the relationship between Noonan syndro...,What is the relationship between Noonan syndro...,polycystic renal disease,OTHER,"[{'aid': 'TQ1A1', 'text': "" Noonan's syndrome ..."
1,TQ2,Gluten information,Re:NDC# 0115-0672-50 Zolmitriptan tabkets 5mg....,Do 5 mg. Zolmitriptan tabkets contain gluten?,Do Zolmitriptan 5mg tablets manufactured by G...,Zolmitriptan,OTHER,"[{'aid': 'TQ2A1', 'text': 'Zolmitriptan tablet..."
2,TQ3,amphetamine salts 20 mg,are they gluten free\t,Are amphetamine salts of 20 mg dosage gluten f...,Do amphetamine salts 20mg tablets contain gluten?,amphetamine salts,OTHER,"[{'aid': 'TQ3A1', 'text': '\n\t\t\t\t\tActive ..."
3,TQ4,vdrl positive,vdrl positive patients please tell me what ar...,What are the treatments and precautions for VD...,What are the treatments and precautions for VD...,vdrl positive,TREATMENT,"[{'aid': 'TQ4A1', 'text': "" Syphilis\n\t\t\t\t..."
4,TQ5,how much glucagon,How much glucose is in my GlucaGen HypoKit ? ...,How much glucagon is in my GlucaGen kit?,How much glucagon is in the GlucaGen HypoKit a...,GlucaGen HypoKit,OTHER,"[{'aid': 'TQ5A1', 'text': ' GLUCAGEN\n\t\t\t\t..."
...,...,...,...,...,...,...,...,...
99,TQ100,General health,how does effextor cause ED and what is the mi...,To what extent does Effexor cause ED?,Could effexor cause ED and in what doses?,effextor,OTHER,"[{'aid': 'TQ100A1', 'text': ' The recommended ..."
100,TQ101,NSAIDS as a potential cause of ED,How long has this non prescription drug been i...,How long has Non-aspirin NSAID been implicated...,Could NSAIDS cause erectile dysfunction?,NSAIDS,OTHER,"[{'aid': 'TQ101A1', 'text': ' Non-aspirin NSAI..."
101,TQ102,General health,i want to know more about aeortic stenosis,"What is aortic stenosis, and is there anything...",What is aortic stenosis?,aeortic stenosis,TREATMENT,"[{'aid': 'TQ102A1', 'text': ' The aorta is the..."
102,TQ103,NaN,What can cause white cells ti uprate,",",What causes increase in white blood cell count?,white cells uprate,INFORMATION,"[{'aid': 'TQ103A1', 'text': ' A high white blo..."


In [17]:
df2['type'] = df2['type'].str.upper()
df2['type'].value_counts()

type
TREATMENT      309
INFORMATION    262
OTHER           63
Name: count, dtype: int64

In [10]:
df['subject'].isna().sum()

1

In [11]:
df2['type'].str.upper()

0           INFORMATION
1           INFORMATION
2             TREATMENT
3             TREATMENT
4      CONTRAINDICATION
             ...       
629           DIAGNOSIS
630           PROGNOSIS
631           TREATMENT
632           TREATMENT
633           TREATMENT
Name: type, Length: 634, dtype: object

In [26]:
labels1 = df['type_info'].unique()
labels2 = df2['type'].unique()

In [9]:
print(f'{labels1}, {labels2}')

['EFFECT' 'INGREDIENT' 'PREVENTION' 'CONTRAINDICATION' 'CAUSE' 'TREATMENT'
 'INFORMATION' 'SYMPTOM' 'DIAGNOSIS' 'PERSON_ORGANIZATION' 'TAPERING'
 'USAGE' 'COMPLICATION' 'INTERACTION' 'SUSCEPTIBILITY' 'ALTERNATIVE'
 'INDICATION' 'STORAGE_DISPOSAL' 'SIDE_EFFECT' 'LIFESTYLE_DIET'
 'COMPARISON' 'PROGNOSIS' 'DOSAGE'], ['INFORMATION' 'TREATMENT' 'CONTRAINDICATION' 'CAUSE' 'SYMPTOM'
 'GENETIC CHANGES' 'INHERITANCE' 'RESOURCES' 'PREVENTION' 'DIAGNOSIS'
 'PROGNOSIS' 'COMPLICATION' 'TAPERING' 'SIDE EFFECTS' 'DOSAGE'
 'ASSOCIATION' 'INDICATION' 'INTERACTION' 'STORAGE AND DISPOSAL'
 'INGREDIENT' 'USAGE' 'ORGANIZATION' 'SUSCEPTIBILITY' 'DIAGNOSE_ME']


In [27]:
labels_igual = []
labels_distintos = []
for i in labels1:
    if i in labels2:
        labels_igual.append(i)
    else:
        labels_distintos.append(i)

In [28]:
labels_distintos2 = []
for j in labels2:
    if j not in labels1:
        labels_distintos2.append(j)

In [29]:
labels_igual

['OTHER', 'TREATMENT', 'CAUSE', 'INFORMATION', 'DIAGNOSIS']

In [30]:
labels_distintos

[]

In [31]:
labels_distintos2

[]

In [17]:
labels1

array(['EFFECT', 'INGREDIENT', 'PREVENTION', 'CONTRAINDICATION', 'CAUSE',
       'TREATMENT', 'INFORMATION', 'SYMPTOM', 'DIAGNOSIS',
       'PERSON_ORGANIZATION', 'TAPERING', 'USAGE', 'COMPLICATION',
       'INTERACTION', 'SUSCEPTIBILITY', 'ALTERNATIVE', 'INDICATION',
       'STORAGE_DISPOSAL', 'SIDE_EFFECT', 'LIFESTYLE_DIET', 'COMPARISON',
       'PROGNOSIS', 'DOSAGE'], dtype=object)

In [12]:
mapeo_tres_etiquetas = {
    ## Dentro de TREATMENT
    'LIFESTYLE_DIET': 'TREATMENT',
    'ALTERNATIVE': 'TREATMENT',
    'PREVENTION': 'TREATMENT',
    'CONTRAINDICATION': 'TREATMENT',
    'TAPERING': 'TREATMENT',
    'DOSAGE': 'TREATMENT',
    
    ## Dentro de INFOTMATION
    'STORAGE_DISPOSAL': 'INFORMATION',
    'STORAGE AND DISPOSAL': 'INFORMATION',
    'CAUSE': 'INFORMATION',
    'GENETIC CHANGES': 'INFORMATION',  
    'INHERITANCE': 'INFORMATION',
    'INDICATION': 'INFORMATION',
    'DIAGNOSIS': 'INFORMATION',
    'DIAGNOSE_ME': 'INFORMATION',
    'PROGNOSIS': 'INFORMATION',
    'SYMPTOM': 'INFORMATION',
    
    ## Dentro de OTHER
    'SIDE EFFECTS': 'OTHER',  
    'SIDE_EFFECT': 'OTHER',
    'EFFECT': 'OTHER',
    'INGREDIENT': 'OTHER',
    'PERSON_ORGANIZATION': 'OTHER',  
    'RESOURCES': 'OTHER',
    'ORGANIZATION': 'OTHER',
    'USAGE': 'OTHER',
    'COMPLICATION': 'OTHER',
    'INTERACTION': 'OTHER',
    'SUSCEPTIBILITY': 'OTHER',
    'COMPARISON': 'OTHER',
    'ASSOCIATION': 'OTHER',
}

In [16]:
df2['type'] = df2['type'].replace(mapeo_tres_etiquetas)
#df['type_info'] = df['type_info'].replace(mapeo_etiquetas)

In [3]:
import re

def clean_text(text: str) -> str:
    # Caracteres especiales de UFT-8
    text = text.encode('utf-8').decode('unicode_escape')
    # Convertir texto a minúsculas
    text = text.lower()
    # Eliminar caracteres especiales y números
    text = re.sub(r'[^a-z]', ' ', text)
    
    return text